In [1]:
import pandas as pd
import pyodbc

# -------------------------
# SQL Server connection
# -------------------------
server = '157.119.230.120,4071'
database = 'PowerBI_LISV2'
username = 'sa'
password = 'Y0m@Sql.!.123'

conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

conn = pyodbc.connect(conn_str)

# -------------------------
# Client list
# -------------------------
x = [
    'AmulManagement','CholayilManagement'
]

# -------------------------
# Store summary info
# -------------------------
summary_data = []

# -------------------------
# Fetch data per client
# -------------------------
for client in x:
    query = f"""
        SELECT *,
                CONCAT(UserName,'-',LISStoreCode,'-',ProductCode,'-',FORMAT(TRY_CONVERT(date, [SubmittedDate&Time], 105), 'dd')) AS UserKey
        FROM {client}.PrimaryShelfStockReportExport
        WHERE MONTH(TRY_CONVERT(date, [SubmittedDate&Time], 105)) = 1 AND Year = '2026';
    """

    df = pd.read_sql(query, conn)

    # Clean column names
    df.columns = df.columns.str.strip()

    # Ensure UserKey exists
    if 'UserKey' not in df.columns or df['UserKey'].isna().all():
        df['UserKey'] = (
            df['UserName'].astype(str) + '-' +
            pd.to_datetime(df['Date'], errors='coerce')
              .dt.day.astype(str).str.zfill(2)
        )

    # Create dynamic DataFrame variable
    df_name = f"{client}_PrimarySelfStock"
    globals()[df_name] = df

    print(f"{df_name} created with {len(df)} rows")

    # Collect summary info
    summary_data.append({
        "ClientName": client,
        "No_of_Rows": len(df)
    })

# -------------------------
# Client-wise summary (ONLY ONCE)
# -------------------------
summary_df = pd.DataFrame(summary_data)
print("\nClient-wise Primary Self Stock Summary:")
print(summary_df)


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41780\3426535272.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


AmulManagement_PrimarySelfStock created with 173631 rows


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_41780\3426535272.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


CholayilManagement_PrimarySelfStock created with 24605 rows

Client-wise Primary Self Stock Summary:
           ClientName  No_of_Rows
0      AmulManagement      173631
1  CholayilManagement       24605
